In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html

from dash.dependencies import Input, Output, State

import dash_cytoscape as cyto

import json

## Add nodes and edges

In [2]:
# !!! important add 'url':'link_to_img' or 'NA'  ,   'classes':img

# nodes and edges
nodes = [
            {'data': {'id': 'one', 'label': 'Node 1', 'url':'https://upload.wikimedia.org/wikipedia/commons/b/b2/Jay_Town_official_portrait.jpg'},
             'classes':'img'},
            {'data': {'id': 'two', 'label': 'Node 2','url':'NA'},'classes':'img'},
            {'data': {'id': '3', 'label': 'Node 3','url':'NA'},'classes':'img'},
            {'data': {'id': '4', 'label': 'Node 4','url':'NA'},'classes':'img'}
            ]
#make juct edges
edges =[    {'data': {'source': 'one', 'target': 'two'}},
            {'data': {'source': 'one', 'target': '3'}},
            {'data': {'source': 'one', 'target': '4'}},
            {'data': {'source': '3', 'target': '4'}}
        ]

In [3]:
# list label val for dropdown-callbacks-label
listLabelVal = list()
for k in nodes[0]['data'].keys():
    if k in ['url']:
        continue
    listLabelVal.append({'label': k, 'value': 'data('+k+')'})

In [4]:
net = nodes + edges

## Styles

In [5]:
# style for network
default_stylesheet = [{
        "selector": 'node',
        'style': {
            'opacity': 0.9,
            'content': 'data(label)'
            #'background-fit': 'cover',
            #'background-image': 'data(img)'
        }
    }, {
        'selector': 'edge',
        'style': {
            'opacity': 0.9,
            "curve-style": "bezier",
            'source-arrow-shape': 'triangle'
        }
    }    
]


## Img Node

In [6]:
style_img_node = [{
        'selector': '.img',
        'style': {
            'background-fit': 'cover',
            'background-image': 'data(url)'
        }
}
    
 ]

In [7]:
# styles for dash
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

## DASH 

In [8]:
# main container
app.layout = html.Div(style={'height':'95vh'},children=[
    
    #container
    html.Div(className='row',
             style = {'width': '100%'},
    #left side
             children=[
    
        html.Div(id='network-block',
                 className="eight columns",
                 style={'border': 'thin lightblue solid','border-radius': 10},
                 children =[
                html.H6('Netwok'),

                cyto.Cytoscape(
                id='cytoscape-net',
                boxSelectionEnabled = True,
                layout={'name': 'random'},
                style={'width': '100%', 'height': '85vh'},
                stylesheet = default_stylesheet,
                elements=net
            )]),
        
        #right side
        html.Div(id='adjust-block',
                 className="four columns",
                 style={'border-radius': 10,
                        'margin-bottom':10 },
                 children =[
                
                
                html.H6('Adjust Layouts'),
                #dropdown layout
                dcc.Dropdown(
                        id='dropdown-callbacks-layout',
                        value='random',
                        clearable=False,
                        options=[
                            {'label': name.capitalize(), 'value': name}
                            for name in ['grid', 'random', 'circle', 'cose', 'concentric']
                        ]
                    ),
                
                #label Nodes
                html.H6('Adjust Label of Node'),
                     
                dcc.Dropdown(
                        id='dropdown-callbacks-label',
                        value='data(label)',
                        clearable=False,
                        options=listLabelVal
                    )

                 ]),
        html.Div(id='node-img-block',
                 className="four columns",
                 style={'backgroundColor':'lightgreen',
                        'border-radius': 10,
                       'height': '500px'},
                 children =[
                #-----------------------
                dcc.Tabs(id="tabs", children=[
                dcc.Tab(label='Info', children = [
                    # show img & previue info
                    html.Div(id='img-prosecutor', style = {'margin':'10px'}, children = [])
                ]),
                    
                dcc.Tab(label='Data', children=[
                    html.Div([html.H6('Info json'),
                    html.Pre(id='output_json',children=[], #)
                         style = {'overflow': 'scroll',
                                     'height': '400px'
                                  #'border': 'thin lightgrey solid'
                                 
                                 })
                    ])
                ])
            ])
                

                 ])
    
    ])
             
])

## Functionals

In [9]:
#Following & Followed nodes and edges
@app.callback([Output('output_json', 'children'),
              Output('cytoscape-net', 'stylesheet'),
              Output('img-prosecutor','children')],

              [Input('cytoscape-net', 'tapNode'),
               Input('cytoscape-net', 'selectedNodeData'),
               Input('dropdown-callbacks-label', 'value')])

def test(node,selectedNodeData,nodeLabel):
    if not node:
        new_styles = [{
            "selector": 'node',
            'style': {
                'opacity': 1,
                'content': nodeLabel,
                #'background-image': 'data(url)'
                
            }
        }]
        return 'Node not chosen', default_stylesheet + new_styles + style_img_node ,html.H6('Node not chosen')
    if selectedNodeData == []:
        new_styles = [{
            "selector": 'node',
            'style': {
                'opacity': 1,
                'content': nodeLabel,
                #'background-image': 'data(url)'
            }
        }]
        return 'Node not chosen', default_stylesheet + new_styles + style_img_node ,html.H6('Node not chosen')
    
    
    if node:   
        stylesheet = [{
            "selector": 'node',
            'style': {
                'opacity': 0.9,
                'content': nodeLabel,
                #'background-image': 'data(url)'
            }
        }, {
            'selector': 'edge',
            'style': {
                'opacity': 0.9,
                "curve-style": "bezier",
                'source-arrow-shape': 'triangle'
            }
        }
        ]
        
        #tapNode change border
        stylesheet.append({
                    "selector": 'node[id = "{}"]'.format(node['data']['id']),
                    "style": {
                        'shape': 'rectangle',
                        'border-width': 6,
                        'border-color': 'yellow',
                        'border-opacity':0.5,
                    }
                })

        #following
        #change color
        following_color = 'red'
        
        for edge in node['edgesData']:
            #edge
            if edge['source'] == node['data']['id']:
                stylesheet.append({
                    "selector": 'node[id = "{}"]'.format(edge['target']),
                    "style": {
                        'background-color': following_color,
                        'opacity': 0.9
                    }
                })
            #node
                stylesheet.append({
                    "selector": 'edge[id= "{}"]'.format(edge['id']),
                    "style": {
                        "line-color": following_color,
                        'source-arrow-color':following_color
                    }
                })
        
        #follower
        #change color 
        follower_color = 'green'
        for edge in node['edgesData']:
            #edge
            if edge['target'] == node['data']['id']:
                stylesheet.append({
                    "selector": 'node[id = "{}"]'.format(edge['source']),
                    "style": {
                        'background-color': follower_color,
                        'opacity': 0.9
                    }
                })
            #node
                stylesheet.append({
                    "selector": 'edge[id= "{}"]'.format(edge['id']),
                    "style": {
                        "line-color": follower_color,
                        'source-arrow-color': follower_color
                    }
                })
        img = html.Div([html.Img(src=node['data']['url'], style={'height':'30%', 'width':'30%'}),
              html.Div(
                  [html.P('Name: '+node['data']['label']),
                  html.P('Office: '+node['data']['id'])]
              )])
              
                
        return json.dumps(node['data'], indent=2), stylesheet + style_img_node ,img 

In [10]:
#change layout
@app.callback(Output('cytoscape-net', 'layout'),
              [Input('dropdown-callbacks-layout', 'value')])
def update_layout(layout):
    return {
        'name': layout,
        'animate': True
    }

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
